In [1]:
import sys
import time
import os
import gc
import argparse
import numpy as np
import pandas as pd
import concurrent
from tqdm import tqdm

sys.path.append('..')
import alntools.density as ds
import alntools as aln
from alntools.postprocess import filter_result_dataframe
from alntools.numeric import embedding_local_similarity

/home/users/kkaminski/anaconda3/envs/plmblast/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
db = "/home/nfs/kkaminski/PLMBLST/ecod70db_20220902"
module = aln.base.Extractor()
module.SIGMA_FACTOR = 1
module.WINDOW_SIZE = 10
module.GAP_OPEN = 0
module.GAP_EXT = 0
module.BFACTOR = 1
module.FILTER_RESULTS = True
head = 50
data = pd.read_csv(db + '.csv').head(head)
dbsize = data.shape[0]
num_combinations = dbsize * (dbsize - 1) // 2
filelist = [os.path.join(db, f'{fileid}.emb') for fileid in range(0, 59990)]
filelist = filelist[:head]
filedict = {i : file for i, file in enumerate(filelist)}
#assert dbsize == len(filelist)
embedding_list = ds.load_full_embeddings(filelist=filelist, poolfactor=4)
embedding_list = [emb.numpy() for emb in embedding_list]
record_stack : list = []
step = 60


batchiter = aln.base.BatchIterator(filedict, 1)
print(len(batchiter))
print('filter', module.FILTER_RESULTS)
print('workers', 1)
print('samples', len(embedding_list), len(filelist))
print('batch size', batchiter.batchsize)
print('num batches per query', len(batchiter))

loading embeddings: 100%|██████████| 50/50 [00:00<00:00, 2392.94it/s]

embedding missing files:  0
50
filter True
workers 1
samples 50 50
batch size 1
num batches per query 50


In [7]:
record_stack = []
for idx, (file, emb) in tqdm(enumerate(zip(filelist, embedding_list))):
        for batchfiles, batchslice in batchiter:
                # submit jobs
            for (idx1, file1), emb1 in zip(batchfiles, embedding_list[batchslice]):
                    iter_id = f'{idx}-{idx1}'
                    job = module.full_compare(emb, emb1, iter_id, file1, 0.2)
                    if any(job):
                        record_stack.append(job)
record_stack = pd.concat(record_stack, axis=0)

50it [00:10,  4.59it/s]


In [8]:
print('mean aln len', record_stack.len.mean())
print('num alns', record_stack.shape[0])

mean aln len 49.77823691460055
num alns 726


In [ ]:
frame = []
for i, emb1 in enumerate(embedding_list):
    for j, emb2 in enumerate(embedding_list):
        if i >= j:
            continue
        density = embedding_local_similarity(emb1, emb2)
        frame.append({'mean' : density.mean(), 'std' : density.std(), 'max' : density.max()})

In [ ]:
frame = pd.DataFrame(frame)

In [ ]:
frame.describe()

,mean,std,max
count,499500.000000,499500.000000,499500.000000
mean,0.024778,0.078055,0.449963
std,0.011696,0.010443,0.103338
min,-0.013246,0.056523,0.184828
25%,0.016910,0.071366,0.373642
50%,0.022968,0.075617,0.437872
75%,0.030532,0.081867,0.513833
max,0.191323,0.228130,0.997478


In [ ]:
density.std()

0.07656269